In [3]:
import pandas as pd


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

column_names = [
    "hotel_name", "city", "country", "price_per_night_text", "price_per_night", "rating", "review_count_text", "review_count", "stars", "location", "amenities"
]

df = pd.read_csv("vietnam_hotels_data.csv", names=column_names, na_values="?")

df.head(200)

,hotel_name,city,country,price_per_night_text,price_per_night,rating,review_count_text,review_count,stars,location,amenities
0,hotel_name,city,country,price_per_night_text,price_per_night,rating,review_count_text,review_count,stars,location,amenities
1,Khach san Muong Thanh Luxury Saigon Hotel,Ho Chi Minh,Vietnam,"₫ 1,411,299",1411299,8.6 Excellent,"5,659 reviews",5659.0,4.0,"Phú Nhuận, Ho Chi Minh City - 3.2 km to center","['Breakfast', 'Free fitness center access', '+5']"
2,Bay Hotel Ho Chi Minh,Ho Chi Minh,Vietnam,"₫ 1,511,618",1511618,8.3 Excellent,"4,635 reviews",4635.0,4.0,"District 1, Ho Chi Minh City - City center","['Breakfast', 'Free fitness center access', '+10']"
3,Belvilla Comfort Stays Near SECC formerly Cozy Hotel,Ho Chi Minh,Vietnam,"₫ 411,545",411545,8.8 Excellent,403 reviews,403.0,4.0,"District 7, Ho Chi Minh City - 3.5 km to center",[]
4,Hotel Continental Saigon,Ho Chi Minh,Vietnam,"₫ 1,987,327",1987327,8.4 Excellent,"7,861 reviews",7861.0,4.0,"District 1, Ho Chi Minh City - City center","['Breakfast', 'Free fitness center access', '+3']"
5,Yoko Airport Saigon Hotel,Ho Chi Minh,Vietnam,"₫ 340,796",340796,8.0 Excellent,"1,464 reviews",1464.0,4.0,"Tân Bình, Ho Chi Minh City - 4.7 km to center","['Parking', 'Express check-in', 'Free WiFi']"
6,LOTTE HOTEL SAIGON,Ho Chi Minh,Vietnam,"₫ 5,399,551",5399551,9.0 Exceptional,"9,953 reviews",9953.0,5.0,"District 1, Ho Chi Minh City - City center",[]
7,Muong Thanh Saigon Centre Hotel,Ho Chi Minh,Vietnam,"₫ 1,665,166",1665166,8.5 Excellent,"3,680 reviews",3680.0,4.0,"District 1, Ho Chi Minh City - City center","['Breakfast', 'Welcome drink', 'Parking', '+3']"
8,Asian Ruby Center Point Hotel,Ho Chi Minh,Vietnam,"₫ 430,545",430545,7.6 Very good,"2,938 reviews",2938.0,3.0,"District 1, Ho Chi Minh City - City center","['Breakfast', 'Welcome drink', 'Parking', 'Free WiFi']"
9,Dazi Signature,Ho Chi Minh,Vietnam,"₫ 1,340,991",1340991,8.7 Excellent,"1,834 reviews",1834.0,4.0,"District 1, Ho Chi Minh City - City center","['Breakfast', 'Parking', 'Express check-in', '+2']"
